In [1]:
import pandas as pd
import numpy as np
from random import randrange, uniform
from sklearn import linear_model

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:1085: UserWarning: Duplicate key in file "/Users/Pani/.matplotlib/matplotlibrc", line #2
  (fname, cnt))


height has been deprecated.



In [3]:

train = pd.read_csv('we_data/train.csv')
test = pd.read_csv('we_data/test.csv')
validation = pd.read_csv('we_data/validation.csv')

In [14]:
def evaluate_metrics(bidprices, display=True):
    win_items = validation['payprice'] < bidprices
    imp = validation[win_items]
    clicks = imp['click'].sum()

    cost = imp['payprice'].sum() / 1000

    avg_CTR = (imp['click'].sum() / imp['click'].count()) * 100
#     Check for CTR

    avg_CPC = cost / imp['click'].sum()
 
    CPM = (cost) / (imp['click'].count()) * 1000
    if display:
        print('number of clicks', clicks)
        print('total cost', cost)
        print('Click through rate ', avg_CTR)
        print('Average CPC', avg_CPC)
        print('Cost per mille', CPM)
    return clicks, cost, avg_CTR

In [4]:
%%time
def preprocess(data):
    result = data['useragent'].str.split('_')
    data['os'] = np.array(result.tolist())[:,0]
    data['browser'] = np.array(result.tolist())[:,1]
    data['slotsize'] = data['slotwidth'] * data['slotheight']
    result = data['usertag'].str.split(',')
    unique_usertags = {elem
                       for elems in result.tolist()
                       for elem in elems}
    for usertag in unique_usertags:
        data['usertag_'+usertag] = result.apply(lambda x: usertag in x).astype(int)

preprocess(validation)
preprocess(test)
preprocess(train)
# train_sample.shape[0]

CPU times: user 1min 51s, sys: 6.53 s, total: 1min 57s
Wall time: 1min 58s


In [5]:
%%time
train_sample = train.sample(n=int(0.1 * (train.shape[0])))
preprocess(train_sample)
print(train_sample.shape)

(243098, 97)
CPU times: user 12.5 s, sys: 8.75 s, total: 21.2 s
Wall time: 22 s


In [277]:
import gc

# algo 2
def freq_ctr_encode(data, column, value=False, one_hot=True):
    ctr_data = train.groupby(column)['click'].mean()
    ctr_data.name = 'ctr'
    freq_data = train.groupby(column)['click'].count()
    freq_data.name = 'freq'
    feat_data = pd.concat([ctr_data, freq_data, ecpc_data], axis=1)
    result = data.join(feat_data, on=column, lsuffix='_' + column)
    result.rename(columns = {'ctr': 'ctr_'+column, 'freq': 'freq_'+column}, inplace=True)
    result_cols = ['ctr_'+column, 'freq_'+column]
    if not value:
        return result[result_cols]
    if one_hot:
        return pd.concat([result[result_cols], 
                          one_hot_encode(data, column)
                         ], axis=1)
    return result[result_cols + [column]]

def freq_ctr_encode_tag(data, tag):
    mask = train[tag] == 1
    tag_data = train[mask]
    ctr_data = tag_data['click'].mean()
    result = pd.DataFrame({'ctr_'+tag: data[tag]})
    result.loc[mask, 'ctr_'+tag] = ctr_data
    return result

def preprocess_super_pctr(data):
    gc.collect()
    pctr_x = data[['weekday', 'hour', 'adexchange', 'advertiser','slotwidth', 'slotheight', 'slotprice']]
    pctr_x.loc[:,'adexchange'] = pd.to_numeric(pctr_x['adexchange'], errors='coerce')
    usertag_df = pd.concat([data[column]
                            for column in data.columns
                            if column.startswith('usertag_')],
                          axis=1)
    usertag_df = ensure_columns(usertag_df, train_usertag_columns)

    pctr_x = pd.concat([pctr_x] + [freq_ctr_encode_tag(usertag_df, column)
                                   for column in usertag_df.columns], axis=1)

    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'weekday')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'hour')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'advertiser')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'adexchange')], axis=1)

    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotvisibility')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotformat')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'city')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'region')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'os')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'browser')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotsize')], axis=1)

    pctr_x.fillna(0, inplace=True)
    return pctr_x

# gc.collect()
preprocess_pctr = preprocess_super_pctr

In [8]:
%%time

pctr_train_x = preprocess_pctr(train_sample)
assert(np.all(np.isfinite(pctr_train_x)))
print(pctr_train_x.shape)
pctr_train_x.head()

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(243098, 541)
CPU times: user 4.75 s, sys: 4.3 s, total: 9.05 s
Wall time: 9.61 s


In [23]:
%%time
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
pctr_train_y = train_sample['click']
clf.fit(pctr_train_x, pctr_train_y)
rf_pctr = clf.predict_proba(pctr_train_x)

CPU times: user 20 s, sys: 2.7 s, total: 22.7 s
Wall time: 23.1 s


In [24]:
%%time
avgCTR = train_sample['click'].mean()
lr = linear_model.LinearRegression(fit_intercept=False)
lr_feat_x = rf_pctr[:,1]/avgCTR
lr_feat_x = lr_feat_x.reshape(-1, 1)
print (lr_feat_x.shape)
print(train_sample['bidprice'].shape)
lr.fit(lr_feat_x, train_sample['bidprice'])

(243098, 1)
(243098,)
CPU times: user 13.6 ms, sys: 16.1 ms, total: 29.7 ms
Wall time: 26.6 ms


In [11]:
%%time
pctr_valid_x = preprocess_pctr(validation)

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


CPU times: user 5.68 s, sys: 5.42 s, total: 11.1 s
Wall time: 11.3 s


In [25]:
%%time
rf_valid_pctr = clf.predict_proba(pctr_valid_x)

CPU times: user 3.2 s, sys: 2.12 s, total: 5.32 s
Wall time: 5.71 s


In [26]:

from sklearn import metrics
import math

results = pd.DataFrame(columns = ['AUC', 'RMSE'])
results.index.name = 'Advertiser'

for advertiser in validation['advertiser'].unique():
    mask = validation['advertiser'] == advertiser
    adv_pctr_valid_x = pctr_valid_x[mask]
    adv_clicks = validation[mask]['click']
    adv_rf_valid_pctr = clf.predict_proba(adv_pctr_valid_x)[:,1]

    adv_auc = metrics.roc_auc_score(adv_clicks, adv_rf_valid_pctr)
    adv_rmse = math.sqrt(metrics.mean_squared_error(adv_clicks, adv_rf_valid_pctr))

    results.loc[advertiser] = [adv_auc, adv_rmse]

    
all_auc = metrics.roc_auc_score(validation['click'], rf_valid_pctr[:,1])
all_rmse = math.sqrt(metrics.mean_squared_error(validation['click'], rf_valid_pctr[:,1]))

results.loc['All'] = [all_auc, all_rmse]

train_auc = metrics.roc_auc_score(train_sample['click'], rf_pctr[:,1])
train_rmse = math.sqrt(metrics.mean_squared_error(train_sample['click'], rf_pctr[:,1]))

results.loc['Train'] = [train_auc, train_rmse]

results

,AUC,RMSE
Advertiser,,
1458,0.865784,0.027866
3476,0.716689,0.016828
3358,0.805764,0.026325
3427,0.824149,0.027123
3386,0.705552,0.022183
2261,0.868630,0.014887
2821,0.571004,0.029450
2259,0.258063,0.010963
2997,0.567545,0.064771


In [28]:
# rf_pctr
# %%time
budget = 6250
best_scale = None
best_clicks = None

for scale in np.linspace(0.1,10.,1000):
    base_bid = lr.coef_ * scale
    xgb_bids = base_bid * rf_valid_pctr[:,1]/avgCTR
    clicks, cost, avg_CTR = evaluate_metrics(xgb_bids, display=False)
#     print('clicks', clicks, scale, base_bid)
    if best_clicks is None or (clicks > best_clicks and cost < budget):
        best_clicks = clicks
        best_scale = scale
        print('new best clicks: num clicks ', best_clicks, ' scale ', best_scale, ' base bid ', lr.coef_, ' cost ', cost)


new best clicks: num clicks  45  scale  0.1  base bid  [ 115.69422137]  cost  262.784
new best clicks: num clicks  46  scale  0.10990990991  base bid  [ 115.69422137]  cost  305.048
new best clicks: num clicks  47  scale  0.11981981982  base bid  [ 115.69422137]  cost  350.335
new best clicks: num clicks  50  scale  0.12972972973  base bid  [ 115.69422137]  cost  393.855
new best clicks: num clicks  51  scale  0.13963963964  base bid  [ 115.69422137]  cost  440.383
new best clicks: num clicks  53  scale  0.14954954955  base bid  [ 115.69422137]  cost  491.256
new best clicks: num clicks  55  scale  0.159459459459  base bid  [ 115.69422137]  cost  537.975
new best clicks: num clicks  59  scale  0.169369369369  base bid  [ 115.69422137]  cost  586.685
new best clicks: num clicks  61  scale  0.179279279279  base bid  [ 115.69422137]  cost  642.197
new best clicks: num clicks  63  scale  0.189189189189  base bid  [ 115.69422137]  cost  695.589
new best clicks: num clicks  70  scale  0.2090

KeyboardInterrupt: 

In [29]:
print(results.round(3).to_latex())

\begin{tabular}{lrr}
\toprule
{} &    AUC &   RMSE \\
Advertiser &        &        \\
\midrule
1458       &  0.866 &  0.028 \\
3476       &  0.717 &  0.017 \\
3358       &  0.806 &  0.026 \\
3427       &  0.824 &  0.027 \\
3386       &  0.706 &  0.022 \\
2261       &  0.869 &  0.015 \\
2821       &  0.571 &  0.029 \\
2259       &  0.258 &  0.011 \\
2997       &  0.568 &  0.065 \\
All        &  0.796 &  0.026 \\
Train      &  0.925 &  0.028 \\
\bottomrule
\end{tabular}

